In [1]:
import requests
from datetime import timedelta, date, datetime
import json
import pandas as pd
import os
import csv
import sys
import time

In [2]:
sys.path.append('../../code')  # Replace '/path/to/code' with the actual path to your 'code' directory
from scrape_new import daterange, get_daily_game_ids

In [3]:
def extract_team_pitching_stats(team_data, team_type, writer, custom_game_id):
    team_stats = team_data.get(team_type, {}).get('statistics', {}).get('pitching', {})
    
    if team_stats:
        # Directly write the stats into the CSV
        writer.writerow({
            'home_away': team_type,
            'ip': team_stats['overall'].get('ip_2', '0'),
            'h': team_stats['overall']['onbase'].get('h', 'N/A'),
            'r': team_stats['overall']['runs'].get('total', 'N/A'),
            'er': team_stats['overall']['runs'].get('earned', 'N/A'),
            'bb': team_stats['overall']['onbase'].get('bb', 'N/A'),
            'so': team_stats['overall']['outs'].get('ktotal', 'N/A'),
            'hr': team_stats['overall']['onbase'].get('hr', 'N/A'),
            'earned_run_avg': team_stats['overall'].get('era', '0'),
            'batters_faced': team_stats['overall'].get('bf', 'N/A'),
            'pitches': team_stats['overall'].get('pitches', {}).get('count', 'N/A'),
            'strikes_total': team_stats['overall'].get('pitches', {}).get('ktotal', 'N/A'),
            'strikes_contact': 'N/A',
            'strikes_swinging': team_stats['overall'].get('outcome', {}).get('kswing', 'N/A'),
            'strikes_looking': team_stats['overall'].get('outcome', {}).get('klook', 'N/A'),
            'inplay_gb_total': team_stats['overall'].get('in_play', {}).get('groundball', 'N/A'),
            'inplay_fb_total': team_stats['overall'].get('in_play', {}).get('flyball', 'N/A'),
            'inplay_ld': team_stats['overall'].get('in_play', {}).get('linedrive', 'N/A'),
            'inplay_unk': 'N/A',
            'game_score': 'N/A',
            'inherited_runners': team_stats['overall'].get('runs', {}).get('ir', 'N/A'),
            'inherited_score': team_stats['overall'].get('runs', {}).get('ira', 'N/A'),
            'wpa_def': 'N/A',
            'leverage_index_avg': 'N/A',
            're24_def': 'N/A',
            'game_id': custom_game_id
        })

In [4]:
def fetch_and_store_pitching_stats(api_key, start_date, delay_seconds=1):
    file_path = '../../data/pitching-test-2.csv'
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    fieldnames = [
        'home_away', 'ip', 'h', 'r', 'er', 'bb', 'so', 'hr', 'earned_run_avg', 'batters_faced', 
        'pitches', 'strikes_total', 'strikes_contact', 'strikes_swinging', 'strikes_looking', 
        'inplay_gb_total', 'inplay_fb_total', 'inplay_ld', 'inplay_unk', 'game_score', 
        'inherited_runners', 'inherited_score', 'wpa_def', 'leverage_index_avg', 're24_def', 'game_id'
    ]

    with open(file_path, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if os.stat(file_path).st_size == 0:
            writer.writeheader()

        for current_date in daterange(start_date):
            year = current_date.year
            month = current_date.month
            day = current_date.day
            game_ids = get_daily_game_ids(api_key, year, month, day)

            for game_id in game_ids:
                url = f'https://api.sportradar.com/mlb/trial/v7/en/games/{game_id}/extended_summary.json?api_key={api_key}'
                response = requests.get(url)

                if response.status_code == 200:
                    data = response.json()
                    game_info = data['game']
                    date = datetime.strptime(game_info['scheduled'], "%Y-%m-%dT%H:%M:%S%z").strftime("%Y%m%d")
                    day_night = game_info['day_night']
                    custom_game_id = f"{game_info['away']['abbr']}{game_info['home']['abbr']}{date}{day_night}"

                    extract_team_pitching_stats(data['game'], 'home', writer, custom_game_id)
                    extract_team_pitching_stats(data['game'], 'away', writer, custom_game_id)

                    time.sleep(delay_seconds)
                else:
                    print(f"Failed to fetch data: {response.status_code} {response.text}")
                    time.sleep(delay_seconds)

        print(f"Team pitching data written to '{file_path}'")

In [6]:
api_key = 'FLs5RpyKo35hLh1QPdYkF4FU91Spgrwn9sNnbyTM'  # Replace with your actual API key
start_date = datetime(2024, 4, 23).date()
fetch_and_store_pitching_stats(api_key, start_date, delay_seconds=1)

Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/23/summary.json | Status Code: 200
Games found on 2024-04-23: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/24/summary.json | Status Code: 200
Games found on 2024-04-24: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/25/summary.json | Status Code: 200
Games found on 2024-04-25: 10
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/26/summary.json | Status Code: 200
Games found on 2024-04-26: 14
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/27/summary.json | Status Code: 200
Games found on 2024-04-27: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/28/summary.json | Status Code: 200
Games found on 2024-04-28: 15
Request URL: https://api.sportradar.com/mlb/trial/v7/en/games/2024/04/29/summary.json | Status Code: 200
Games found on 2024-04-29: 12
Request URL: https://api.sportradar.com/mlb/trial/v7/en